In [ ]:
# python -m venv policyengine_venv
# source policyengine_venv/bin/activate
# make sure to select kernel to be in the above environment
# Install ipykernel if not already installed
!pip install ipykernel
!pip install python-dotenv
# Register the environment as a kernel
!python -m ipykernel install --user --name=policyengine_venv --display-name="Python (PolicyEngine)"
# import sys
# print(sys.executable)  # Should show a path inside your policyengine_venv directory
#Install policy engine
!pip install policyengine-uk

In [ ]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Access the tokens
token = os.environ.get("HUGGING_FACE_TOKEN")

# Now you can use the token
print(f"Token loaded: {token[:4]}...{token[-4:]}")  # Prints first and last 4 chars for verification

In [ ]:
from policyengine_uk import Microsimulation
from policyengine_uk.model_api import *
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
from policyengine_core.parameters import Parameter

import pandas as pd
# LOCAL_enhanced_frs_2022_23="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5"
# sim = Microsimulation(dataset=LOCAL_enhanced_frs_2022_23)
#sim = Microsimulation(dataset="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5")
 

In [ ]:
# Level 1: 20 GBP pw uplift to child element of UC
start_date= "2029-01-01"
eval_year=2029

def change_tax_parameters(parameters):
    add=20.00*52/12
    parameters.gov.dwp.universal_credit.elements.child.amount.update(
        start=instant(start_date), value=orig_param+add
        )
    parameters.gov.dwp.universal_credit.elements.child.first.higher_amount.update(
        start=instant(start_date), value=orig_param_higher+add
    )
    return parameters


class reform(Reform):
    def apply(self):
        #self.modify_parameters(change_tax_parameters)
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
Microsimulation()
orig_param = baseline.tax_benefit_system.parameters.gov.dwp.universal_credit.elements.child.amount(start_date)
orig_param_higher = baseline.tax_benefit_system.parameters.gov.dwp.universal_credit.elements.child.first.higher_amount(start_date)
print(orig_param,orig_param_higher )
reformed = Microsimulation(reform=reform)

revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"This reform would cost £{revenue / 1e9:.2f}bn in {eval_year}")
povertycalc(baseline, eval_year, reformed)
avg_gain_gainer(baseline, reformed, eval_year)


In [ ]:
#Level1b:  Work allowance increased to 1000 GBP  pa
start_date= "2029-01-01"
eval_year=2029

def change_tax_parameters(parameters):
    add=1000.00/12  # Monthly increase

    parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing.update(
        start=instant(start_date), value=orig_without_housing+add
    )
    parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing.update(
        start=instant(start_date), value=orig_with_housing+add
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
orig_without_housing = baseline.tax_benefit_system.parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing(start_date)
orig_with_housing = baseline.tax_benefit_system.parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing(start_date)
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calc("gov_spending", eval_year).sum()
print(f"This reform would cost £{revenue / 1e9:.2f}bn in {eval_year}")

povertycalc(baseline, eval_year, reformed)
avg_gain_gainer(baseline, reformed, eval_year)



In [ ]:
#Level 1c: Change child limit
start_date= "2026-01-01"
eval_year=2026
def change_tax_parameters(parameters):
    parameters.gov.dwp.universal_credit.elements.child.limit.child_count.update(
        start=instant(start_date), value=inf
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
povertycalc(baseline, eval_year, reformed)
avg_gain_gainer(baseline, reformed, eval_year)


In [ ]:
#Level 2b UNDER 5s premium
from policyengine_uk import Microsimulation
from policyengine_uk.reforms.uc_bonus_for_under5 import reform
eval_year=2029
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
print(reformed.calculate("num_households_with_under5", eval_year).sum())
print(reformed.calculate("universal_credit", eval_year).mean())
print(baseline.calculate("universal_credit", eval_year).mean())
print(f"Number of households with at least one child under 5: {under5_households}")
revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
povertycalc(baseline, eval_year, reformed)  
avg_gain_gainer(baseline, reformed, eval_year)

In [ ]:
#Level 3 ADD 30 HOUR WORK BONUS
from policyengine_uk import Microsimulation
from policyengine_uk.reforms.uc_30hour_bonus_v2 import reform
from eval_functions import povertycalc, avg_gain_gainer
eval_year=2026
baseline = Microsimulation()
NMW = baseline.tax_benefit_system.parameters.gov.hmrc.minimum_wage.OVER_24(eval_year)
print(NMW)
reformed = Microsimulation(reform=reform)
# Debug: Check if UC is actually changing
baseline_uc = baseline.calculate("universal_credit", eval_year)
reformed_uc = reformed.calculate("universal_credit", eval_year)
print("baseline uc:",baseline.calculate("universal_credit", eval_year).mean())
print("reformed uc:",reformed.calculate("universal_credit", eval_year).mean())
revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
povertycalc(baseline, eval_year, reformed)  
avg_gain_gainer(baseline, reformed, eval_year)

In [ ]:
# Add EITC phase in
from policyengine_uk import Microsimulation
from policyengine_uk.reforms.uc_phase_in import reform
from eval_functions import povertycalc, avg_gain_gainer
eval_year=2026
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
# Debug: Check if UC is actually changing
baseline_uc = baseline.calculate("universal_credit", eval_year)
reformed_uc = reformed.calculate("universal_credit", eval_year)
print("baseline uc:",baseline.calculate("universal_credit", eval_year).mean())
print("reformed uc:",reformed.calculate("universal_credit", eval_year).mean())

revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
povertycalc(baseline, eval_year, reformed)  
avg_gain_gainer(baseline, reformed, eval_year)



baseline uc: 1911.24199084687
reformed uc: 1977.077014808142
Cost: £2.6bn
Reduction in child poverty: 114,519 in 2026, reformed
Avg gain of gainers 2026: 1696.5663441974175
